In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


In [6]:
from pydantic import BaseModel
from typing import List
class Restaurant(BaseModel):
    store_name: List[str]
    score: List[str]
    address: List[str]
    business_status: List[str]

class RestaurantList(BaseModel):
    restaurants: List[Restaurant]

In [7]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Local Restaurant Researcher",
    goal="Search all restaurants within 1km of the user's location, and search for the information in detail and accurately based on Google Maps [store name, score, address, current business status]",
    backstory="""
    The role of a local restaurant researcher has been assigned. Your mission is to search for restaurant information on Google within a 1 km radius of the current location of the provided user. Follow these instructions correctly:
    It uses information from multiple sources and always double-checks whether the source is true or up to date.

    1. **Use user location only:** Only use the provided user location (e.g., "Seongbuk-gu Jeongneung-ro 77") as a reference for your search. Do not use other locations or default to overseas locations.
    2. **Expanding Google Search: ** Leverages Google Maps and other trusted Google data sources to find as many restaurants as possible within the designated 1 km area.
    3. **Extract data:** Extract and check the following details for each restaurant:
        - **Restaurant name:** The exact name displayed on Google Maps.
        - **Rated/scored: ** Currently displayed rating or score.
        - **Exact street address:** Get the correct street name and number (use a modern street address format rather than a traditional site-based address).
        - **Business status:** Check if the restaurant is currently open or closed.
        - **Menu info: **Sell what menus are sold, and research what's popular.
    4. **Comprehensive Collection: ** Gather as detailed and accurate information as possible, relying solely on the location provided by you.
    5. **Output from the following agent:** Compile all restaurant details in a structured format and forward them to the next agent for further processing.

    Follow these guidelines thoroughly to ensure accurate and complete data extraction.
    You want to impress your colleagues with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[search_tool, scrape_tool],
    max_iter=1,
)

selector = Agent(
    role="Verified Restaurant Selector",
    goal="Filter and select the best 5 restaurants that match the user's preferences while ensuring accuracy and reliability.",
    backstory="""
    You are a restaurant recommendation expert who can analyze a user's taste and choose a restaurant that sells food similar to that taste.
    It serves as a recommendation for restaurants that sell food similar to the user's taste.
    You are working for a very important client.
    """,
    verbose=True,
)

search_task = Task(
    description="""
    Perform a highly accurate search for real restaurants within a strict 1km radius of "{user_location}".
    Use Google search to find results from maps.google.com and verify:
    - Store name
    - Accurate Google rating
    - Address (must be in road name format)
    - Business status (Open or Closed)
    - Menu info(Sell what menus are sold, and research what's popular)
    """,
    agent=researcher,
    expected_output="Verified list of real restaurants from Google Maps.",
)

scrape_task = Task(
    description="""
    Scrape the Google search results to extract:
    - Verified restaurant names
    - Accurate Google Maps rating
    - Correct road name format address
    - Business status (Open or Closed)
    - Menu info(Sell what menus are sold, and research what's popular)
    Ensure all extracted data corresponds with actual Google Maps listings.
    """,
    agent=researcher,
    expected_output="Highly accurate restaurant information scraped from Google search results.",
)

selection_task = Task(
    description="""
    From the provided restaurant data, carefully select exactly 5 restaurants that best match the user's preferences: "{user_preferences}".
    Ensure that only verified, real restaurants from Google Maps are included in the final selection.
    
    Format the output as structured JSON:
    [
        {{
            "store_name": "Restaurant Name",
            "score": "Star Rating",
            "address": "Accurate Road Address",
            "business_status": "Open or Closed"
        }},
        ...
    ]
    """,
    agent=selector,
    expected_output="Only real, existing restaurants structured in JSON format.",
    output_pydantic=RestaurantList,
)

crew = Crew(
    agents=[researcher, selector],
    tasks=[search_task, scrape_task, selection_task],
    verbose=True,
)

user_location = "성북구 정릉로 77"
user_preferences = "한식, 매운 음식"

# CrewAI 실행
result = crew.kickoff(
    inputs={
        "user_location": user_location,
        "user_preferences": user_preferences,
    }
)

output_json = result.model_dump_json(indent=4)
with open("recommended_restaurants.json", "w", encoding="utf-8") as f:
    f.write(output_json)

2025-03-20 20:21:44,893 - 8690403392 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Local Restaurant Researcher
 [INFO]: == Starting Task: 
    Perform a highly accurate search for real restaurants within a strict 1km radius of "성북구 정릉로 77".
    Use Google search to find results from maps.google.com and verify:
    - Store name
    - Accurate Google rating
    - Address (must be in road name format)
    - Business status (Open or Closed)
    - Menu info(Sell what menus are sold, and research what's popular)
    


> Entering new CrewAgentExecutor chain...
Action: Search the internet  
Action Input: {"search_query": "restaurants near 성북구 정릉로 77"}   


Search results: Title: 공차국민대점 - 트리플
Link: https://triple.guide/restaurants/e6b8f579-8505-4c45-be41-97db867db466
Snippet: 공차국민대점 · 주소. 서울특별시 ﻿성북구 정릉로 77, 지상1층 (정릉동, 국민대학교3동2호관, 북악관) · 전화. +8229104952.
---
Title: 내가 먹어본 정릉(or 성북구) 맛집 : 네이버 블로그
Link: https://blog.naver.com/PostView.nhn?blogId=ebp9595&logNo=223626659615&redirect=Dlog&widgetTypeCall=true
Snippet: 골목식당 정릉편에 나왔던 곳! 보국문역 근처로 자리 옮기시고 나서

/Users/parksanghyeok/Desktop/sanghyeok/capstone-2025-21/AI/env/lib/python3.11/site-packages/crewai/task.py:275: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  converter = Converter(


 [DEBUG]: == [Verified Restaurant Selector] Task output: restaurants=[Restaurant(store_name=['Pizza Place'], score=['4.5'], address=['123 Main St, Cityville, ST 12345'], business_status=['Open']), Restaurant(store_name=['Sushi Spot'], score=['4.8'], address=['456 Elm St, Cityville, ST 12345'], business_status=['Open']), Restaurant(store_name=['Burger Joint'], score=['4.2'], address=['789 Oak St, Cityville, ST 12345'], business_status=['Closed'])]


